### Load in and clean up data

In [8]:
import pandas as pd
from sodapy import Socrata

In [13]:
client = Socrata("health.data.ny.gov", None)

# First 50,000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("tg3i-cinn", limit=50000)

# Convert to pandas DataFrame
nyd2019_50k = pd.DataFrame.from_records(results)
nyd2019_50k.shape

(50000, 33)

In [ ]:
#create length of stay variable

In [ ]:
#go from county to latitiude longitude

### Create regression models

In [ ]:
# predict total charges and length of stay from demographic variables

### Create streamlit app